> reference: https://huggingface.co/docs/transformers/tasks/sequence_classification

In [ ]:
import pandas as pd
# train_df = pd.read_csv("train.csv")
titles = pd.read_csv('../input/upppm/titles.csv')

In [ ]:
input_path = '../input/us-patent-phrase-to-phrase-matching/'

In [ ]:
import pandas as pd
# input_path = '/content/gdrive/My Drive/patent/input/'
train_df = pd.read_csv(f"{input_path}train.csv")
test_df = pd.read_csv(f"{input_path}test.csv")



In [ ]:
import random
import numpy as np
import torch

seed_val = 13

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("../input/bert-for-patent/bert-for-patents")

## US Patent dataset

In [ ]:
# tokenizer.save_pretrained('./pre_trained_tokenizer/distilbert-base-uncased')

In [ ]:
train_df.sample(3)

In [ ]:
titles.sample(3)

In [ ]:
train_df.shape

In [ ]:
titles.shape

In [ ]:
titles[titles['code'].str.contains('H05')]

In [ ]:
titles = titles.drop_duplicates(subset=['title'])

In [ ]:
titles.shape

In [ ]:
#titles['code_short'] = titles['code'].str[:3]
#titles['code_aggr'] = titles[['title','code_short']].groupby(['code_short'])['title'].transform(lambda x: '; '.join(x))
#titles = titles.drop_duplicates(subset=['code'])
#.strip("{|}")
#titles = titles[['code_short','code_aggr']]


In [ ]:
titles.shape

In [ ]:
train_df = train_df.merge(titles,left_on='context', right_on='code')


In [ ]:
train_df.shape

In [ ]:
train_df.sample(3)

In [ ]:
#train_df = train_df[["anchor","target","score","code_aggr"]]
#train_df = train_df.drop(columns=['id', 'context', 'code_short'])


In [ ]:
train_df.sample(3)

In [ ]:
#train_df = train_df.merge(titles, left_on='context', right_on='code')
train_df['text'] = train_df.apply(lambda x: f"{x['anchor']} [SEP] {x['title']} [SEP] {x['target']}", axis=1)

print(len(train_df))

In [ ]:
def clean_titles(text):
    text = text.lower()
    text = text.replace('[{}]','')
    return text
 

In [ ]:
train_df['text'] = train_df['text'].transform(clean_titles)

In [ ]:
titles.sample(3)

In [ ]:
tokenized_text = tokenizer.batch_encode_plus(train_df["text"].tolist(), truncation=False,return_tensors="pt",padding=True)['input_ids']

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer


model = AutoModelForSequenceClassification.from_pretrained("../input/bert-for-patent/bert-for-patents", num_labels=6)

In [ ]:
# model.save_pretrained('./pre_trained_model/distilbert-base-uncased')

In [ ]:
#type(tokenized_imdb['train'])

In [ ]:
train_df.sample(3)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
le.fit(train_df['score'].to_list())
print(le.classes_)

In [ ]:
# split
from sklearn.model_selection import train_test_split

X, y = tokenized_text, le.transform(train_df['score'].to_list())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed_val)

In [ ]:
del train_df

In [ ]:
y_test[0:10]

In [ ]:
print(X_train[0:2])

In [ ]:
batch_size = 32

In [ ]:
from torch.utils.data import TensorDataset
import torch

# to_numpy = lambda x: torch.from_numpy(X_train)

train_inputs = torch.tensor(X_train, dtype=torch.int)
train_labels = torch.tensor(y_train, dtype=torch.int)
validation_inputs = torch.tensor(X_test, dtype=torch.int)
validation_labels = torch.tensor(y_test, dtype=torch.int)
print(train_labels.shape,X_train.shape)
print(validation_labels.shape,X_test.shape)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, WeightedRandomSampler

train_data = TensorDataset(train_inputs, train_labels )
validation_data = TensorDataset(validation_inputs, validation_labels)



In [ ]:
class_sample_count = np.array(
    [len(np.where(train_labels == t)[0]) for t in np.unique(train_labels)])
weight = 1. / class_sample_count


In [ ]:
samples_weight = np.array([weight[t] for t in train_labels])

samples_weight = torch.from_numpy(samples_weight)
samples_weigth = samples_weight.double()
weighted_sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

In [ ]:
#train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, sampler=weighted_sampler, batch_size=batch_size)

validation_sampler = SequentialSampler(validation_data)
validation_loader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:

output_dir = '/content/gdrive/My Drive/patent/distil-bert'
def save_model(model,output_dir):
  if not os.path.exists(output_dir):
      os.makedirs(output_dir)

  model_to_save = model.module if hasattr(model, 'module') else model  
  model_to_save.save_pretrained(output_dir)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)


In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 6

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_loader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


In [ ]:
import random
import time
import os

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128




# Store the average loss after each epoch so we can plot them.
loss_values = []
epochs =7 
for epoch_i in range(0, epochs):
    t0 = time.time()
    total_loss = 0
    model.train()
    for step, batch in enumerate(train_loader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)            
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_loader), elapsed))

        b_input_ids = batch[0].long().to(device)
        b_labels = batch[1].long().to(device)

        model.zero_grad()        
        outputs = model(b_input_ids, labels=b_labels)        
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_loader)            
    loss_values.append(avg_train_loss)


    t0 = time.time()
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    
    for batch in validation_loader:        
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_labels = batch
        b_labels = b_labels.long()
        b_input_ids = b_input_ids.long()
        with torch.no_grad():        
            outputs = model(b_input_ids)
        

        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    save_model(model, output_dir)
print("")
print("Training complete!")

In [ ]:
merged_test = test_df.merge(titles,left_on='context', right_on='code')

merged_test['text'] = merged_test.apply(lambda x: f"{x['anchor']} [SEP] {x['title']} [SEP] {x['target']}", axis=1)
tokenized_text = tokenizer.batch_encode_plus(merged_test["text"].tolist(), truncation=False,return_tensors="pt",padding=True)['input_ids']
X_test = tokenized_text
test_inputs = torch.tensor(X_test)
test_data = TensorDataset(test_inputs)

test_sampler = SequentialSampler(test_data)
test_loader = DataLoader(test_data, sampler=test_sampler, batch_size=32)



In [ ]:
print(len(X_test))

In [ ]:
model.eval()

predictions_test = []

for batch in test_loader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids = batch[0]
    b_input_ids = b_input_ids.long()
    with torch.no_grad():
        outputs = model(b_input_ids)

    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    
    predictions_test.append(logits)

print('    DONE.')


In [ ]:
import numpy
predictions_test = numpy.concatenate( predictions_test, axis=0 )

In [ ]:
#predictions_test

In [ ]:
import scipy
pred_labels=[]
for i in range(len(predictions_test)):
    probs=scipy.special.softmax(predictions_test[i])
    pred_labels_i = np.argmax(predictions_test[i], axis=0).flatten()
    pred_labels.append(pred_labels_i[0])

In [ ]:
le.inverse_transform(pred_labels)

In [ ]:
submission_df = pd.DataFrame({"id":merged_test["id"].tolist(), "score":le.inverse_transform(pred_labels)})

In [ ]:
submission_df

In [ ]:
submission_df.to_csv('submission.csv',index=False)

In [ ]:
!cat submission.csv